<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/%ED%98%84%EC%A3%BC/melon_kakao_word2vec_train%20n%20val%20n%20test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

Cloning into 'khaiii'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 957 (delta 25), reused 49 (delta 19), pack-reused 877
Receiving objects: 100% (957/957), 33.03 MiB | 25.11 MiB/s, done.
Resolving deltas: 100% (383/383), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from collections import Counter
from typing import *
from khaiii import KhaiiiApi

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
folder = '/content/gdrive/My Drive/멜론 플레이리스트'
filelist = os.chdir(folder)

In [8]:
genre_gn_all=pd.read_json('genre_gn_all.json', typ='series')
train=pd.read_json('train.json')
val = pd.read_json('val.json')
test = pd.read_json('test.json')

In [ ]:
!git clone https://github.com/slipaway/melonplaylist.git

# 0. 함수 정의

In [7]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def uppertolower(string):
  if isEnglish(string) == True:
    string = string.lower() #태그나 제목이 영어로 들어간 경우, 모두 소문자로 바꿔주기

In [73]:
def re_sub(series: pd.Series) -> pd.Series:
  series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True) #ㅋ 제거용
  series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True) #특수문자 제거용
  series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True) #공백 제거
  series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True) #u3000제거
  return series

def flatten(list_of_list : List) -> List:
  flatten = [j for i in list_of_list for j in i]
  return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:

  if len(title)==0 or title== ' ' : #제목이 공백인 경우 tokenizer 에러 발생
    return []
  
  result = tokenizer.analyze(title)
  result = [(morph.lex, morph.tag) for split in result for morph in split.morphs] # (형태소, 품사) 튜플의 리스트
  return result

def get_all_tags(df) -> List:
  tag_list = df['tags'].values.tolist()
  tag_list=flatten(tag_list)
  return tag_list

# 1. 태그 전처리

In [ ]:
 train['tags'].head(10)

In [ ]:
#영어는 소문자로 모두 바꾸기
for i in range(len(train['tags'])):
  for j in range(len(train['tags'][i])):
    train['tags'][i][j]= train['tags'][i][j].lower()

In [85]:
tokenizer = KhaiiiApi()
all_tag = get_all_tags(train)
token_tag = [get_token(x, tokenizer) for x in all_tag] #태그를 형태소 분석

In [ ]:
token_tag[:40]

In [90]:
token_itself=list(filter(lambda x: len(x)==1, token_tag)) # 태그 자체가 형태소여서 분리되지 않는 태그만 고름
token_itself=flatten(token_itself)
flatten_token=flatten(token_tag)

In [91]:
print('%-23s'%'# of original tag is', f'{len(all_tag):8,}')
print('%-23s'%'# of morpheme itself is', f'{len(token_itself):8,}')
print('%-23s'%'# of total token is', f'{len(flatten_token):8,}')

# of original tag is     476,331
# of morpheme itself is  333,093
# of total token is      722,860


In [92]:
#counting part of speech (pos)
pos=[x[1] for x in token_itself]
pos_count=Counter(pos)
popular_pos = pos_count.most_common()

In [ ]:
#tag 분류표
objects=[x[0] for x in popular_pos]
y_pos=np.arange(len(objects))
performance=[x[1] for x in popular_pos]

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Usage')
plt.title('Part of Speech - Tags')
plt.show()

# 2. 플레이리스트 제목 전처리

In [ ]:
#플레이리스트 제목 형태소 분석

train['plylst_title']=re_sub(train['plylst_title'])
train.loc[:, 'ply_token']=train['plylst_title'].map(lambda x: get_token(x, tokenizer))

In [ ]:
using_pos=['NNG', 'SL', 'NNP', 'MAG', 'SN'] #일반 명사, 외국어, 고유 명사, 일반 부사, 숫자
train['ply_token']=train['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))


In [95]:
unique_tag=set(token_itself)
unique_word=[x[0] for x in unique_tag]

#train 데이터의 plylst title 형태소 분리

In [ ]:
#정답 tag에 나온 형태소만 남겨두기
train['ply_token']=train['ply_token'].map(lambda x: list(filter(lambda x: x[0] in unique_word, x)))
train.head(10)

# 3-1. 태그: word2vec 임베딩 with train dataset



In [9]:
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

class EpochSaver(CallbackAny2Vec):
  def __init__(self, path_prefix):
    self.path_prefix = path_prefix
    self.epoch = 0

  def on_epoch_end(self, model):
    output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
    model.save(output_path)
    self.epoch += 1

class EpochLogger(CallbackAny2Vec):
     def __init__(self):
         self.epoch = 0

     def on_epoch_begin(self, model):
         print("Epoch #{} start".format(self.epoch))

     def on_epoch_end(self, model):
         print("Epoch #{} end".format(self.epoch))
         self.epoch += 1

In [10]:
tag_list = [train['tags'][i] for i in range(len(train['tags']))]
epoch_logger = EpochLogger()
tag_embedding_model = Word2Vec(tag_list, window = 3, min_count=3, workers=4, iter=1000, sg=1, callbacks=[epoch_logger])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end
Epoch #30 start
Epoch #30 end
Epoch #31 start
Epoch #31 end
Epoch #32 start
Epoch #32 end
Epoch #33 start
Epoch #33 end


In [11]:
list(tag_embedding_model.wv.vocab.keys())

['락',
 '추억',
 '회상',
 '까페',
 '잔잔한',
 '연말',
 '눈오는날',
 '캐럴',
 '분위기',
 '따듯한',
 '크리스마스캐럴',
 '겨울노래',
 '크리스마스',
 '겨울왕국',
 '크리스마스송',
 '댄스',
 '운동',
 '드라이브',
 'Pop',
 '트로피컬하우스',
 '힐링',
 '기분전환',
 '2017',
 '팝',
 '트렌드',
 '일렉',
 '짝사랑',
 '취향저격',
 '슬픔',
 '고백',
 '사랑',
 '이별',
 '일렉트로니카',
 '포크',
 '메탈',
 '인디',
 '록',
 'Metal',
 'M에센셜',
 'Rock',
 'kpop',
 '걸그룹댄스',
 '스트레스해소',
 '새해',
 '여행',
 '프로필음악',
 '카카오톡',
 '소원',
 '프로필',
 '소망',
 '다짐',
 '카톡',
 '듣고',
 '우울',
 '힘내',
 '힙합',
 '느낌있는',
 '밤',
 '새벽',
 'RnB',
 '감각적인',
 '국내',
 '그루브한',
 '가을',
 '재즈',
 '감성',
 '질리지않는',
 '나만알고싶은',
 '봄',
 '설렘',
 '비오는날',
 '스밍',
 '목록',
 '폐막식',
 '올림픽',
 '엑소',
 '조용히',
 '혼자',
 '새벽감성',
 '고민',
 '맥주한잔',
 '카페',
 'OST',
 '어쿠스틱',
 '편안한',
 '에너지',
 '듀엣',
 '달달',
 '피쳐링',
 '남녀',
 '신인',
 '국힙',
 '신나는',
 '존윌리엄스',
 '영화클래식',
 '겨울',
 '아침',
 '빅밴드',
 '상쾌한',
 '밝은',
 '스윙',
 '스윙재즈',
 '휴식',
 '여름',
 '매장음악',
 '헬스',
 '스포츠',
 '피트니스',
 '다이어트',
 '런닝',
 '필라테스',
 '산책',
 '요가',
 '발라드',
 '캐롤',
 '위로',
 '80년대',
 '90년대발라드',
 '올드송',
 '90년대',
 '옛날노래',
 '월드',
 '연주',
 '클럽',
 '스트레스',
 '주

예시.

In [ ]:
tag_embedding_model.wv.most_similar('연말')

In [ ]:
tag_embedding_model.wv.most_similar('조용')

In [ ]:
tag_embedding_model.wv.most_similar(positive = ['감성','기분전환'])

#4-1. 곡: word2vec 임베딩 with train dataset

In [ ]:
train['songs']

In [102]:
#integer를 string으로 바꿔주기
for i in range(len(train['songs'])):
  for j in range(len(train['songs'][i])):
    train['songs'][i][j]= str(train['songs'][i][j])

In [103]:
song_list = [train['songs'][i] for i in range(len(train['songs']))]

In [104]:
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

In [105]:
class EpochSaver(CallbackAny2Vec):
...     '''Callback to save model after each epoch.'''
...
...     def __init__(self, path_prefix):
...         self.path_prefix = path_prefix
...         self.epoch = 0
...
...     def on_epoch_end(self, model):
...         output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
...         model.save(output_path)
...         self.epoch += 1

class EpochLogger(CallbackAny2Vec):
     def __init__(self):
         self.epoch = 0

     def on_epoch_begin(self, model):
         print("Epoch #{} start".format(self.epoch))

     def on_epoch_end(self, model):
         print("Epoch #{} end".format(self.epoch))
         self.epoch += 1

In [106]:
from gensim.models import Word2Vec
epoch_logger = EpochLogger()
song_embedding_model = Word2Vec(song_list, size=400, window = 5, min_count=2, workers=4, iter=100, sg=1, callbacks=[epoch_logger])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end
Epoch #30 start
Epoch #30 end
Epoch #31 start
Epoch #31 end
Epoch #32 start
Epoch #32 end
Epoch #33 start
Epoch #33 end


In [107]:
list(song_embedding_model.wv.vocab.keys())

['525514',
 '129701',
 '562083',
 '297861',
 '139541',
 '351214',
 '650298',
 '531057',
 '205238',
 '660493',
 '461973',
 '72552',
 '50104',
 '432406',
 '675945',
 '497066',
 '120377',
 '389529',
 '244277',
 '461062',
 '696302',
 '442765',
 '532114',
 '586541',
 '33389',
 '244000',
 '692078',
 '37741',
 '645653',
 '571802',
 '200183',
 '61435',
 '204499',
 '41749',
 '129258',
 '413920',
 '117205',
 '6546',
 '152422',
 '602724',
 '425946',
 '173634',
 '631268',
 '97749',
 '395416',
 '103741',
 '181101',
 '472144',
 '414721',
 '75801',
 '315216',
 '192882',
 '383960',
 '548636',
 '83116',
 '276692',
 '166267',
 '186301',
 '354465',
 '256598',
 '233195',
 '666852',
 '686560',
 '556426',
 '142974',
 '331878',
 '195141',
 '32017',
 '617795',
 '396532',
 '623704',
 '516930',
 '300104',
 '176874',
 '443513',
 '471385',
 '35784',
 '153029',
 '336743',
 '203558',
 '348801',
 '454550',
 '394031',
 '195524',
 '540149',
 '287984',
 '440773',
 '100335',
 '556301',
 '655561',
 '534818',
 '695032',
 

예시.

In [ ]:
song_embedding_model.wv.most_similar('108757') #그렇게 너를 사랑해 - 자전거 탄 풍경 / 포크.블루스

In [ ]:
song_embedding_model.wv.most_similar('700444') #둘이 하나(Love Theme) - 린 / OST, 발라드

In [ ]:
song_embedding_model.wv.most_similar('109526') #선인장 - 에피톤 프로젝트

# 태그 예측하기

# i) 본래 있는 태그를 활용해서 예측

In [ ]:
#태그가 있는 플레이리스트 인덱스 뽑아내기
tag_yu_index = []
tag_no_index = []
for i in range(len(val['tags'])):
  if len(val['tags'][i]) != 0:
    tag_yu_index.append(i)
  else:
    tag_no_index.append(i)

In [ ]:
val_tag_yu = val.loc[tag_yu_index,:]
val_tag_yu = val_tag_yu.reset_index() #index reset
val_tag_yu.head(10)

In [ ]:
val_tag_list = [val_tag_yu['tags'][i] for i in range(len(val_tag_yu['tags']))]

a) train에서 학습하지 못한 tag는 제외 (하나라도 없으면 제외시킴)

In [ ]:
index_list = []
for i in range(len(val_tag_list)):
  for j in range(len(val_tag_list[i])):
    if val_tag_list[i][j] not in list(tag_embedding_model.wv.vocab.keys()):
      index_list.append(i)
index_list = set(index_list)
index_list = list(index_list)

In [ ]:
index_list.sort()
index_list #train에 없는 tag index

In [ ]:
list_for_list = []
val_predicted_tag = []
for i in range(len(val_tag_list)):
  if i in index_list:
    val_predicted_tag.append([])
  else:
    for j in range(10):
      list_for_list.append(tag_embedding_model.wv.most_similar(positive = val_tag_list[i])[j][0])
    val_predicted_tag.append(list_for_list)
    list_for_list = []

In [ ]:
val_tag_yu['predicted_tags']=val_predicted_tag

In [ ]:
val_tag_yu.head(10)

b) 하나라도 train에 있는 것

In [ ]:
val_tag_yu_two = val_tag_yu.iloc[index_list]
val_tag_yu_two

In [ ]:
val_tag_list_two = []
for i in index_list:
  val_tag_list_two.append(val_tag_yu_two['tags'][i])

val_tag_list_two

In [ ]:
index_list_two = []
for i in range(len(val_tag_list_two)):
  for j in range(len(val_tag_list_two[i])):
    if val_tag_list_two[i][j] in list(tag_embedding_model.wv.vocab.keys()):
      index_list_two.append(i)

index_list_two = set(index_list_two)
index_list_two = list(index_list_two)
index_list_two #한번이라도 train에 나왔던 단어의 인덱스

In [ ]:
list_for_list = []
val_predicted_tag = []
for i in range(len(val_tag_list_two)):
  if i not in index_list_two:
    val_predicted_tag.append([])
  else:
    for j in range(len(val_tag_list_two[i])):
      if val_tag_list_two[i][j] not in list(tag_embedding_model.wv.vocab.keys()): continue
      for k in range(10):
        list_for_list.append(tag_embedding_model.wv.most_similar(positive = val_tag_list_two[i][j])[k][0])
    val_predicted_tag.append(list_for_list)
    list_for_list = []

In [ ]:
val_tag_yu_two['predicted_tags']=val_predicted_tag
val_tag_yu_two

In [ ]:
val_tag_yu.iloc[index_list] = val_tag_yu_two
val_tag_yu #태그를 활용해서 태그 뽑기_최종_최종

In [ ]:
val_tag_yu.to_csv('val_tag_yu.csv', header=False, index=False)

# ii) 태그가 없을 때 태그 뽑기

In [ ]:
index_list = []
for i in range(len(val_tag_yu)):
  if len(val_tag_yu['predicted_tags'][i]) == 0:
    index_list.append(i)
index_list

In [ ]:
val_tag_no = val.loc[tag_no_index,:]
val_tag_yu_two = val.loc[index_list,:]
val_tag_no = val_tag_no.reset_index() #index reset
pd.concat([val_tag_no, val_tag_yu_two])
#... 어떻게 하지,,,,,,, ㅇ<-< 믿습니다 성환님

# 곡 예측하기
# i) 플레이리스트에 곡이 있는 경우

In [108]:
#곡이 있는 플레이리스트 인덱스 뽑아내기
song_yu_index = []
song_no_index = []
for i in range(len(val['songs'])):
  if len(val['songs'][i]) != 0:
    song_yu_index.append(i)
  else:
    song_no_index.append(i)

In [109]:
#노래 있는 경우
val_song_yu = val.loc[song_yu_index,:]
val_song_yu = val_song_yu.reset_index()

In [116]:
val_song_yu

,index,tags,id,plylst_title,songs,like_cnt,updt_date
0,0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
2,3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
3,4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
4,5,"[비, 분위기, 컨트리, 서정적인]",138538,,"[173033, 138237, 63140, 15733, 461780, 308519,...",2,2019-06-12 17:43:35.000
...,...,...,...,...,...,...,...
18631,23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
18632,23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
18633,23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
18634,23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [125]:
# train에 없는 노래들 빼기
val_song_list = [list(filter(lambda x: x in list(song_embedding_model.wv.vocab.keys()), val_song_yu['songs'][i])) for i in range(100)]

In [128]:
list_for_list = []
val_predicted_song = []
for i in range(len(val_song_list)):
  if len(val_song_list[i]) == 0:
    val_predicted_song.append([])
  else:
    for j in range(100):
      list_for_list.append(song_embedding_model.wv.most_similar(positive=val_song_list[i], topn=100)[j][0])
      print(i)
    val_predicted_song.append(list_for_list)
    list_for_list = []

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
51
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
53
5

In [112]:
#train에서 학습되지 않은 곡 리스트로 만들기
index_list = []
for i in range(len(val_song_yu['songs'])):
  for j in range(len(val_song_yu['songs'][i])):
    if val_song_yu['songs'][i][j] not in list(song_embedding_model.wv.vocab.keys()):
      index_list.append(val_song_yu['songs'][i][j])

KeyboardInterrupt: ignored

In [20]:
index_list_two = set(index_list)
index_list_two = list(index_list_two)
index_list_two

['647390',
 '399110',
 '429774',
 '177824',
 '429582',
 '251947',
 '566106',
 '198827',
 '620868',
 '378402',
 '694974',
 '444349',
 '322261',
 '621532',
 '178744',
 '380462',
 '32419',
 '593736',
 '587920',
 '59064',
 '17005',
 '502198',
 '636597',
 '706171',
 '704534',
 '397102',
 '111853',
 '41632',
 '636683',
 '688249',
 '404270',
 '135533',
 '363276',
 '459050',
 '489905',
 '202977',
 '278799',
 '336188',
 '548552',
 '508880',
 '694588',
 '689458',
 '124277',
 '257725',
 '437354',
 '250303',
 '234931',
 '450094',
 '390163',
 '537893',
 '284198',
 '417067',
 '578409',
 '339359',
 '487894',
 '186559',
 '149755',
 '377424',
 '528202',
 '619205',
 '238839',
 '300232',
 '381944',
 '564458',
 '269294',
 '549739',
 '58413',
 '421668',
 '307982',
 '462479',
 '154584',
 '424566',
 '258457',
 '305729',
 '272720',
 '568778',
 '500521',
 '570998',
 '147784',
 '153551',
 '599392',
 '119611',
 '149283',
 '211537',
 '707096',
 '140344',
 '318283',
 '645499',
 '43202',
 '567583',
 '128886',
 '285

In [21]:
#train에 없는 곡 제외시키고 리스트 안의 리스트 형태로 만들기
val_song_list = [list(filter(lambda x: x not in index_list_two, val_song_yu['songs'][i])) for i in range(len(val_song_yu['songs']))]

In [25]:
aa = []
for i in range(len(val_song_list)):
  if len(val_song_list[i]) == 0:
    aa.append(i)

In [ ]:
val_predicted_song = []

In [30]:
list_for_list = []
val_predicted_song = []
for i in range(10):
  if i in aa:
    val_predicted_song.append([])
  else:
    for j in range(100):
      list_for_list.append(song_embedding_model.wv.most_similar(positive=val_song_list[i], topn=100)[j][0])
      print(j)
    val_predicted_song.append(list_for_list)
    list_for_list = []

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


KeyboardInterrupt: ignored

In [ ]:
val_predicted_song

In [ ]:
val_song_yu['predicted_songs']=val_predicted_song
val_song_yu

In [ ]:
val_song_yu.to_csv('val_song_yu.csv', header=False, index=False)

# Test: tag 있는 것들 중에서 tag 예측

In [202]:
#태그가 있는 플레이리스트 인덱스 뽑아내기
tag_yu_index = []
tag_no_index = []
for i in range(len(test['tags'])):
  if len(test['tags'][i]) != 0:
    tag_yu_index.append(i)
  else:
    tag_no_index.append(i)

In [203]:
test_tag_yu = test.loc[tag_yu_index,:]
test_tag_yu = test_tag_yu.reset_index()

In [204]:
test_tag_yu

,index,tags,id,plylst_title,songs,like_cnt,updt_date
0,1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
1,2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
2,3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
3,5,[스트레스],88862,노래방 안에서,[],22,2016-01-22 20:55:12.000
4,7,[사랑],133975,봄바람과 듣고싶은 설레는 노래,[],0,2020-02-19 03:05:49.000
...,...,...,...,...,...,...,...
5417,10733,[별],77688,,"[93507, 324019, 221377, 656426, 26083, 661209,...",8,2018-08-10 20:14:19.000
5418,10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000
5419,10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000
5420,10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000


In [154]:
test_tag_list = [list(filter(lambda x: x in list(tag_embedding_model.wv.vocab.keys()), test_tag_yu['tags'][i])) for i in range(len(test_tag_yu['tags']))]

In [155]:
list_for_list = []
val_predicted_tag = []
for i in range(len(test_tag_list)):
  if len(test_tag_list[i]) == 0:
    val_predicted_tag.append([])
  else:
    for k in range(10):
      list_for_list.append(tag_embedding_model.wv.most_similar(positive = test_tag_list[i])[k][0])
    val_predicted_tag.append(list_for_list)
    print(i)
    list_for_list = []

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
295
296
297
298
300
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
411
412
413
414
415
416
417
418
419
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519


In [156]:
test_tag_yu['predicted_tags']=val_predicted_tag
test_tag_yu

,index,tags,id,plylst_title,songs,like_cnt,updt_date,predicted_tags
0,1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000,"[피쉬슈즈, 피터팬컴플렉스, 소수빈, 인디스트리트, 케이인디차트, yasisi, 미..."
1,2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000,"[기분전환, 여행, 빵빵, 운전, 신나는, 흥, 신나는드라이브, 휴게소, 운전할때,..."
2,3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000,"[분위기있는, 어두움, 레트로감성, 비밥, 관능적, 매혹, 감성, 분위기를, 패션위..."
3,5,[스트레스],88862,노래방 안에서,[],22,2016-01-22 20:55:12.000,"[내적댄스, 기분전환, 시원시원, 신나는노래모음, 프로야구, 드라아브, 국내EDM,..."
4,7,[사랑],133975,봄바람과 듣고싶은 설레는 노래,[],0,2020-02-19 03:05:49.000,"[진선미, 반함, 달달송, 상사병, 첫만남, 프로포즈송, 생각했던대로, 남여, 설렌..."
...,...,...,...,...,...,...,...,...
5417,10733,[별],77688,,"[93507, 324019, 221377, 656426, 26083, 661209,...",8,2018-08-10 20:14:19.000,"[달빛, 밤하늘, 선선한바람, 여름밤, 가을밤, 달, 달밤, 무중력, 여수밤바다, 밤]"
5418,10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000,"[회상, 이별은, 담시, 슈가맨시즌2, 올드감성, 재발견, 토토가, 첫이별, 토토가..."
5419,10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000,"[산책해요, 아쿠스틱, 우울하고, 띵_곡, 오후의, 공감하는, 혼자, 생각을, 운전..."
5420,10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000,"[드라이브, 신나는, 여행, 핫한노래, 스트레스, 제주여행, 팝, 청량한, 내적댄스..."


In [158]:
a = test_tag_yu['id']
b = test_tag_yu['predicted_tags']
c = pd.concat([a, b], axis=1)
c.to_json('test_tag_yu3.json') #json 파일로 저장

# 성환 오빠: tag 10개 안 되는 거 10개 만들어주기

In [180]:
tag_addition = pd.read_json('곡만 존재하는 플레이리스트에서 predicted_tags 출력.json')
tag_addition

,id,ply_token,tags,songs,predicted_songs,predicted_tags
0,70107,[],[],"[398985, 449403, 411543, 528044, 143048, 98020...",[],"[황현, 프리스타일, Apink, 고등학생]"
1,102395,[],[],"[528550, 285114, 506667, 339507, 653012, 435029]",[],"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep..."
2,42586,[],[],"[538867, 513110, 225168, 211527, 425074, 21709...",[],"[weeflow, kxan]"
3,13244,[],[],"[32042, 98472, 475222, 56796, 662086, 666787, ...",[],"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피..."
4,55399,[],[],"[264320, 605720]",[],"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso..."
...,...,...,...,...,...,...
4293,120081,[],[],"[224607, 672096, 174950, 202906, 562039, 37070...",[],"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW..."
4294,16489,[],[],"[260730, 243178, 19411, 311420, 55791, 406782,...",[],"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트..."
4295,46470,[],[],"[67655, 65672, 210647, 608386, 413837, 566468,...",[],"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ..."
4296,48788,[],[],"[145292, 402779, 452316, 324334, 217189, 42301...",[],"[가요리믹스, 김경호, 지아]"


In [166]:
not_tag_list_ida = [list(filter(lambda x: x in list(tag_embedding_model.wv.vocab.keys()), tag_addition['predicted_tags'][i])) for i in range(len(tag_addition['predicted_tags']))]

In [ ]:
list_for_list = []
test_predicted_tag = []
for i in range(len(not_tag_list_ida)):
  if len(not_tag_list_ida[i]) == 0:
    test_predicted_tag.append([])
  else:
    for k in range(10):
      list_for_list.append(tag_embedding_model.wv.most_similar(positive=not_tag_list_ida[i])[k][0])
    test_predicted_tag.append(list_for_list)
    print(i)
    list_for_list = []

In [181]:
tag_addition['additional_tag']= test_predicted_tag
tag_addition

,id,ply_token,tags,songs,predicted_songs,predicted_tags,additional_tag
0,70107,[],[],"[398985, 449403, 411543, 528044, 143048, 98020...",[],"[황현, 프리스타일, Apink, 고등학생]","[서지음, 오늘의추천가수, 슴스테, Monotree, 잼팩토리, 모노트리, eone..."
1,102395,[],[],"[528550, 285114, 506667, 339507, 653012, 435029]",[],"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep...","[UMFKOREA2017, 바운드, 울트라뮤직페스티벌, ElectroHouse, 매..."
2,42586,[],[],"[538867, 513110, 225168, 211527, 425074, 21709...",[],"[weeflow, kxan]","[Empire, 히팝, 제국, 센힙합, SMTM, nb2, 홍대클럽홍대엔비, 일리네..."
3,13244,[],[],"[32042, 98472, 475222, 56796, 662086, 666787, ...",[],"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피...","[VINXEN, 딥플로우, 사이먼도미닉, 키비, SIMONDOMINIC, SMTM,..."
4,55399,[],[],"[264320, 605720]",[],"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso...","[만화영화, 애니ost, 중음, 짱구, TV만화주제가, 그시절우리가좋아했던소녀, 쿵..."
...,...,...,...,...,...,...,...
4293,120081,[],[],"[224607, 672096, 174950, 202906, 562039, 37070...",[],"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW...","[본조비, 고스트스테이션, MMFD, ji, mymadfatdiary, ChuckB..."
4294,16489,[],[],"[260730, 243178, 19411, 311420, 55791, 406782,...",[],"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트...","[슴스테, 잼팩토리, 서지음, 방구석1열, EXO_CBX, 재현, 엔시티의나잇나잇,..."
4295,46470,[],[],"[67655, 65672, 210647, 608386, 413837, 566468,...",[],"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ...","[빈자리, 후폭풍, 가슴아픈노래, 이별후유증, 아프다, 리수, 우울할, 돋보이는, ..."
4296,48788,[],[],"[145292, 402779, 452316, 324334, 217189, 42301...",[],"[가요리믹스, 김경호, 지아]","[플라이투더스카이, MCTHEMAX, 노래방인기곡, 이별후에_듣는노래, 김종국, 슬..."


In [182]:
for i in range(len(tag_addition['predicted_tags'])):
  a = len(tag_addition['predicted_tags'][i])
  if a != 10:
    b = tag_addition['additional_tag'][i][0:10-a]
    tag_addition['predicted_tags'][i] = tag_addition['predicted_tags'][i] + b

In [186]:
a = tag_addition['id']
b = tag_addition['predicted_tags']
c = pd.concat([a,b], axis=1)
c.to_json('성환오빠 채운거.json')

In [237]:
c

,id,predicted_tags
0,70107,"[황현, 프리스타일, Apink, 고등학생, 서지음, 오늘의추천가수, 슴스테, Mo..."
1,102395,"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep..."
2,42586,"[weeflow, kxan, Empire, 히팝, 제국, 센힙합, SMTM, nb2..."
3,13244,"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피..."
4,55399,"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso..."
...,...,...
4293,120081,"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW..."
4294,16489,"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트..."
4295,46470,"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ..."
4296,48788,"[가요리믹스, 김경호, 지아, 플라이투더스카이, MCTHEMAX, 노래방인기곡, 이..."


# Test: tag와 제목이 없는 노래들에서 노래 예측하기

In [187]:
#integer를 string으로 바꿔주기
for i in range(len(test['songs'])):
  for j in range(len(test['songs'][i])):
    test['songs'][i][j]= str(test['songs'][i][j])

In [206]:
test_tag_no = test.loc[tag_no_index,:]
test_tag_no = test_tag_no.reset_index()
test_tag_no #태그가 없는 것들

,index,tags,id,plylst_title,songs,like_cnt,updt_date
0,0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000
2,6,[],42586,,"[538867, 513110, 337317, 225168, 211527, 42507...",3,2020-03-13 22:42:27.000
3,8,[],13244,,"[32042, 98472, 475222, 56796, 662086, 666787, ...",15,2020-03-04 21:04:44.000
4,10,[],55399,,"[148079, 651833, 388619, 264320, 347391, 49002...",22,2015-08-26 15:00:16.000
...,...,...,...,...,...,...,...
5313,10730,[],120081,,"[249220, 88841, 359359, 224607, 298719, 262021...",21,2014-09-15 12:31:17.000
5314,10731,[],16489,,"[260730, 31312, 250159, 374880, 243178, 19411,...",39,2017-06-19 23:15:09.000
5315,10732,[],46470,,"[67655, 65672, 210647, 608386, 413837, 566468,...",1,2020-02-03 08:22:06.000
5316,10734,[],48788,,"[145292, 402779, 452316, 324334, 217189, 42301...",1896,2012-11-27 04:38:08.000


In [244]:
title_no_index = []
for i in range(len(test_tag_no)):
  if len(test_tag_no['plylst_title'][i]) == 0:
    title_no_index.append(i)

In [265]:
test_title_tag_no = test_tag_no.loc[title_no_index,:]
test_title_tag_no = test_title_tag_no.reset_index()
test_title_tag_no #태그X, 타이틀X

,level_0,index,tags,id,plylst_title,songs,like_cnt,updt_date
0,0,0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,1,4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000
2,2,6,[],42586,,"[538867, 513110, 337317, 225168, 211527, 42507...",3,2020-03-13 22:42:27.000
3,3,8,[],13244,,"[32042, 98472, 475222, 56796, 662086, 666787, ...",15,2020-03-04 21:04:44.000
4,4,10,[],55399,,"[148079, 651833, 388619, 264320, 347391, 49002...",22,2015-08-26 15:00:16.000
...,...,...,...,...,...,...,...,...
4504,5313,10730,[],120081,,"[249220, 88841, 359359, 224607, 298719, 262021...",21,2014-09-15 12:31:17.000
4505,5314,10731,[],16489,,"[260730, 31312, 250159, 374880, 243178, 19411,...",39,2017-06-19 23:15:09.000
4506,5315,10732,[],46470,,"[67655, 65672, 210647, 608386, 413837, 566468,...",1,2020-02-03 08:22:06.000
4507,5316,10734,[],48788,,"[145292, 402779, 452316, 324334, 217189, 42301...",1896,2012-11-27 04:38:08.000


,level_0,index,tags,id,plylst_title,songs,like_cnt,updt_date
0,0,0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,1,4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000
2,2,6,[],42586,,"[538867, 513110, 337317, 225168, 211527, 42507...",3,2020-03-13 22:42:27.000
3,3,8,[],13244,,"[32042, 98472, 475222, 56796, 662086, 666787, ...",15,2020-03-04 21:04:44.000
4,4,10,[],55399,,"[148079, 651833, 388619, 264320, 347391, 49002...",22,2015-08-26 15:00:16.000
...,...,...,...,...,...,...,...,...
4504,5313,10730,[],120081,,"[249220, 88841, 359359, 224607, 298719, 262021...",21,2014-09-15 12:31:17.000
4505,5314,10731,[],16489,,"[260730, 31312, 250159, 374880, 243178, 19411,...",39,2017-06-19 23:15:09.000
4506,5315,10732,[],46470,,"[67655, 65672, 210647, 608386, 413837, 566468,...",1,2020-02-03 08:22:06.000
4507,5316,10734,[],48788,,"[145292, 402779, 452316, 324334, 217189, 42301...",1896,2012-11-27 04:38:08.000


In [275]:
aa = []
for i in range(len(test_title_tag_no['songs'])):
  if len(test_title_tag_no['songs'][i]) != 0:
    aa.append(i)

In [279]:
test_title_tag_no2 = test_title_tag_no.loc[aa,:]
test_only_song = pd.concat([test_title_tag_no2['id'], test_title_tag_no2['songs']],axis=1)
test_only_song.reset_index()
test_only_song

,id,songs
0,70107,"[398985, 449403, 411543, 528044, 143048, 98020..."
1,102395,"[630683, 481582, 528550, 285114, 506667, 17922..."
2,42586,"[538867, 513110, 337317, 225168, 211527, 42507..."
3,13244,"[32042, 98472, 475222, 56796, 662086, 666787, ..."
4,55399,"[148079, 651833, 388619, 264320, 347391, 49002..."
...,...,...
4504,120081,"[249220, 88841, 359359, 224607, 298719, 262021..."
4505,16489,"[260730, 31312, 250159, 374880, 243178, 19411,..."
4506,46470,"[67655, 65672, 210647, 608386, 413837, 566468,..."
4507,48788,"[145292, 402779, 452316, 324334, 217189, 42301..."


# CV

In [325]:
import math

train_cv=train.iloc[:round(train.shape[0]/10*9), :]
test_cv=train.iloc[round(train.shape[0]/10*9):, :]
test_cv_remove=train.iloc[round(train.shape[0]/10*9):, :]

test_cv_remove=test_cv_remove.astype('object')
for i in range(test_cv_remove.shape[0]):
  test_cv_remove.iloc[i, 3]=test_cv_remove.iloc[i, 3][:math.ceil(len(test_cv_remove.iloc[i, 3])/2)]

In [3]:
cv = test_cv_remove.reset_index()
cv['songs']

NameError: ignored

In [2]:
cv['songs'][0]

NameError: ignored

In [ ]:
cv_list = [list(filter(lambda x: x in list(song_embedding_model.wv.vocab.keys()), cv['songs'][i])) for i in range(len(cv['songs']))]

In [1]:
cv_list

NameError: ignored

In [331]:
list_for_list = []
cv = []
for i in range(len(cv_list)):
  if len(cv_list[i]) == 0:
    cv.append([])
  else:
    for j in range(100):
      list_for_list.append(song_embedding_model.wv.most_similar(positive=cv_list[i], topn=100)[j][0])
    print(i)
    cv.append(list_for_list)
    list_for_list = []

0


KeyError: ignored

# test_only_song 가지고 곡 예측하기 (0, 200)

In [281]:
test_song_list1 = [list(filter(lambda x: x in list(song_embedding_model.wv.vocab.keys()), test_only_song['songs'][i])) for i in range(0, 200)]

In [307]:
list_for_list = []
test_predicted_song = []
for i in range(len(test_song_list1)):
  if len(test_song_list1[i]) == 0:
    test_predicted_song.append([])
  else:
    for j in range(100):
      list_for_list.append(song_embedding_model.wv.most_similar(positive=test_song_list1[i], topn=100)[j][0])
    print(i)
    test_predicted_song.append(list_for_list)
    list_for_list = []

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
34
35
36
37
38
39
40
41
42
43
44
45
46
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
94
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [308]:
a = test_only_song['id'][0:200]
b = pd.Series(test_predicted_song)
c = pd.concat([a,b],axis=1)
c.to_json('songonly1.json')

# test_only_song 가지고 곡 예측하기 (200, 400)

In [293]:
test_song_list2 = [list(filter(lambda x: x in list(song_embedding_model.wv.vocab.keys()), test_only_song['songs'][i])) for i in range(200, 400)]

In [310]:
list_for_list = []
test_predicted_song2 = []
for i in range(len(test_song_list2)):
  if len(test_song_list2[i]) == 0:
    test_predicted_song2.append([])
  else:
    for j in range(100):
      list_for_list.append(song_embedding_model.wv.most_similar(positive=test_song_list2[i], topn=100)[j][0])
    print(i)
    test_predicted_song2.append(list_for_list)
    list_for_list = []

0
1
2
3


KeyboardInterrupt: ignored

In [305]:
c

,id,0
0,70107,"[235353, 506380, 485619, 491989, 360156, 12038..."
1,102395,"[164138, 48893, 24876, 570466, 491577, 705340,..."
2,42586,"[432236, 668531, 517654, 114270, 491659, 26743..."
3,13244,"[272625, 380819, 20052, 17301, 90568, 415579, ..."
4,55399,"[642132, 442215, 322642, 488726, 483686, 52976..."
...,...,...
195,68689,"[265972, 492819, 497372, 634670, 442111, 28270..."
196,26361,"[140376, 67295, 355784, 635449, 354058, 406734..."
197,33661,"[162283, 682444, 251374, 152298, 596766, 61902..."
198,109719,"[610377, 410800, 63840, 642100, 500724, 558722..."


In [303]:
a = test_only_song['id'][0:400]
b = test_predicted_song + test_predicted_song2
d = pd.Series(b)
c = pd.concat([a,d], axis=1)
c

200    150491
201     78605
202    142271
203     98743
204     59611
        ...  
395     22967
396     14341
397     76976
398    111194
399     27188
Name: id, Length: 200, dtype: int64